In [40]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen
from unidecode import unidecode

import os
import re
import pickle
import json

In [2]:
def get_all_ufc_cards():
    cards_links = []
    for page in range(20,0,-1):
        link = urlopen(f'https://www.tapology.com/fightcenter?group=ufc&page={page}&region=&schedule=results')
        site = BeautifulSoup(link,'html.parser')
        links_on_page = []
        for result in site.find_all('section',class_="fcListing"):
            elem = result.find('span','name').a
            links_on_page.append(("https://www.tapology.com" + elem.get('href'),elem.text))        
        [cards_links.append(card) for card in links_on_page[::-1]]
    return cards_links

In [3]:
cards_links = get_all_ufc_cards()

In [4]:
def get_all_ufc_involved(card_list):
    list_of_fighters = []
    checked_fighters = []
    for card_link,card_name in card_list:
        link = urlopen(card_link)
        site = BeautifulSoup(link,'html.parser')
        for bout in site.find_all('li','fightCard'):
            for name in bout.find_all('a'):
                if re.match('/fightcenter/bouts/',name.get('href')) or name.text in checked_fighters: continue
                list_of_fighters.append("https://www.tapology.com" + name.get('href'))
                checked_fighters.append(name.text)
    return list(set(list_of_fighters))

In [5]:
list_of_fighters = get_all_ufc_involved(cards_links)

In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [12]:
binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\firefox.exe')
driver = webdriver.Firefox(firefox_binary=binary)

In [2]:
all_links = pickle.load(open('fighters.p','rb'))

In [16]:
def get_score(string):
    if not string: return {'nc':'0','wins':'0', 'loses':'0','draws':'0'}
    score = {'nc':'0'}
    score['wins'], score['loses'], score['draws'] = re.search('\d+-\d+-\d+',string)[0].split('-')
    if re.search('NC',string):
        score['nc'] = re.search('(\d+) NC',"16-4-0, 1 NC (Win-Loss-Draw)")[1]
    return score

def get_last_fight_date(string):
    months = {'January': '01','February': '02','March': '03',
          'April': '04','May': '05','June': '06',
          'July': '07','August': '08','September': '09',
          'October': '10','November': '11','December': '12'}
    month = months[re.search('\w+',string)[0]]
    day, year = list(re.findall('\d+',string))
    return f'{year}-{month}-{day}'

In [17]:
def get_fighters_info(url):   
    driver.implicitly_wait(2)
    driver.get(url)
    
    element = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, "fighterRecordControls")))
    element.find_element_by_class_name('right').click()
#   driver.find_element_by_id('fighterRecordControls').find_element_by_class_name('right').click()
    site = BeautifulSoup(driver.page_source,'html.parser')
    
    fighter = {'Name':'',
       'Nickame':'',
       'Birth date':'',
       'Given name':'',
       'Height':'',
       'Reach':'',
       'Record':{'wins':'','loses':'','draws':'','nc':''},
       'Last fight':'',
       'Affiliation':'',
       'Born':'',
       'Fighting out of':'',
       'bouts':[]}
    

    fighter['Name'] = unidecode(site.find('div','fighterUpcomingHeader').find_all('h1')[1].string)
    fighter['Nickame'] = '' if not site.find('div','fighterUpcomingHeader').find('h4','nickname').string else site.find('div','fighterUpcomingHeader').find('h4','nickname').string.strip()[1:-1]

    for row in site.find('div','details details_two_columns').find_all('li'):
        for strong_text,plain_text in zip(row.find_all('strong'),row.find_all('span')):
            strong_text, plain_text = strong_text.string, plain_text.string
            if re.search('Given Name:',strong_text): fighter['Given name'] = plain_text
            if re.search('Pro MMA Record:',strong_text): fighter['Record'] = get_score(plain_text)
            if re.search('Date of Birth',strong_text): fighter['Birth date'] = plain_text.replace('.','-')
            if re.search('Last Fight:',strong_text): fighter['Last fight'] = get_last_fight_date(plain_text)
            if re.search('Affiliation:',strong_text): fighter['Affiliation'] = plain_text
            if re.search('Height:',strong_text) and re.search('\((.*)cm\)',plain_text): fighter['Height'] = re.search('\((.*)cm\)',plain_text)[1]
            if re.search('Reach:',strong_text)  and re.search('\((.*)cm\)',plain_text): fighter['Reach'] = re.search('\((.*)cm\)',plain_text)[1]
            if re.search('Born:',strong_text): fighter['Born'] = plain_text
            if re.search('Fighting out of:',strong_text): fighter['Fighting out of'] = plain_text
    
    for fight in site.find('section',class_='fighterFightResults').find_all('li')[::-1]:
        
        bout_info = {
            'Billing':'',
            'Class weight':'',
            'Decision':'',
            'Duration':'',
            'Fight under':'',
            "Fighter's record":'',
            'Injuries':'',
            'Odds':'',
            "Opponent's name":'',
            "Opponent's record":'',
            'Pay':'',
            'Referee':'',
            'Round':'',
            'Time':'',
            'Title in a fight':'',
            'Way':'',
            'Weigh-In':'',
            'Weightclass':'',
            'Weightclass title name':'',
            'Win or Lose':'',
            'fight date':''}
        
        
        if (fight.find('div','record nonMma') or 
            re.search('Cancelled Bout', fight.find('div','lead').text) or 
            re.search('Confirmed Upcoming Bout', fight.find('div','lead').text) or 
            fight.find('div',{'data-result':'cancelled'})): continue

        bout_info["Opponent's name"] = unidecode(fight.find('div','name').text)
        
        if fight.find('span',{"title":'Fighter Record Before Fight'}):
            bout_info["Fighter's record"] = get_score(fight.find('span',{"title":'Fighter Record Before Fight'}).text)
            del bout_info["Fighter's record"]['nc']
        else:
            bout_info["Fighter's record"] = {'wins':'0','loses':'0','draws':'0'}
            
        if fight.find('span',{"title":'Opponent Record Before Fight'}):
            bout_info["Opponent's record"] = get_score(fight.find('span',{"title":'Opponent Record Before Fight'}).text)
            del bout_info["Opponent's record"]['nc']

        if '·' in fight.find('div','lead').text:
            win_info = fight.find('div','lead').text.split('·')
            if len(win_info) == 2:
                bout_info["Win or Lose"] =  win_info[0]
                bout_info["Way"] =  win_info[1]                
            elif len(win_info) == 3:
                bout_info["Win or Lose"] =  win_info[0]
                bout_info["Way"] =  win_info[1]
                bout_info["Decision"] = win_info[2]
            else:
                bout_info["Win or Lose"] = win_info[0]
                bout_info["Way"] = win_info[1]
                bout_info["Time"] = win_info[2]
                bout_info['Round'] = win_info[3]
        else:
            bout_info["Win or Lose"] = fight.find('div','lead').text
        
        bout_info["Fight under"] = fight.find('div','notes').text
        bout_info["fight date"] = fight.find('div','date').text.replace('.','-')

        if fight.find('div',class_='detail tall'):
            for additional_info in fight.find('div',class_='detail tall').find_all('div'):
                if additional_info.find_all('span'):
                    first, second = list(additional_info.find_all('span'))

                    if first.text == 'Title Bout:': 
                        if '·' in second.text:
                            bout_info['Title in a fight'], bout_info['Weightclass title name'] = second.text.split('·')
                        else: 
                            bout_info['Weightclass title name'] = second.text
                    
                    if first.text == 'Billing:': bout_info['Billing'] = second.text
                    if first.text == 'Duration:':
                        if re.search("(.*\d)",second.text)[0]:
                            bout_info['Duration'] = re.search("(.*\d)",second.text)[0]
                        else:
                            bout_info['Duration'] = second.text
                        

                    if first.text == 'Weight:':
                        for weightclass_info in second.text.split('·'):
                            if not re.search("\d",weightclass_info):
                                bout_info['Weightclass'] = weightclass_info

                            elif re.search("Weigh-In",weightclass_info,re.IGNORECASE):
                                bout_info['Weigh-In'] = re.search('(\d+\.?\d? lbs)',weightclass_info)[0]
                                
                            else:
                                bout_info['Class weight'] = re.search('(\d+\.?\d? lbs)',weightclass_info)[0] if re.search('(\d+\.?\d? lbs)',weightclass_info) else ''


                    if first.text == 'Odds:': bout_info['Odds'] = second.text.split('·')[0]
                    if first.text == 'Referee:': bout_info['Referee'] = second.text
                    if first.text == 'Disclosed Pay:': bout_info['Pay'] = re.search('\$(\d+) ',second.text.replace(',',''))[1]
                    if first.text == 'Injuries:': bout_info['Injuries'] = second.text
        
        fighter['bouts'].append(bout_info)
    
    return fighter

In [25]:
list_of_fighters+=["https://www.tapology.com/fightcenter/fighters/antonio-rogerio-nogueira-minotoro"]

In [18]:
get_fighters_info("https://www.tapology.com/fightcenter/fighters/stipe-miocic")

{'Name': 'Stipe Miocic',
 'Nickame': '',
 'Birth date': '1982-08-19',
 'Given name': '',
 'Height': '194',
 'Reach': '203',
 'Record': {'nc': '0', 'wins': '18', 'loses': '3', 'draws': '0'},
 'Last fight': '2018-07-07',
 'Affiliation': 'Strong Style Fight Team',
 'Born': 'Euclid, Ohio, United States',
 'Fighting out of': 'Independence, Ohio',
 'bouts': [{'Billing': '',
   'Class weight': '',
   'Decision': '',
   'Duration': '',
   'Fight under': 'North American Allied Fight Series',
   "Fighter's record": {'wins': '0', 'loses': '0', 'draws': '0'},
   'Injuries': '',
   'Odds': '',
   "Opponent's name": 'Corey  Mullis',
   "Opponent's record": {'wins': '0', 'loses': '1', 'draws': '0'},
   'Pay': '',
   'Referee': '',
   'Round': '',
   'Time': '',
   'Title in a fight': '',
   'Way': '',
   'Weigh-In': '',
   'Weightclass': '',
   'Weightclass title name': '',
   'Win or Lose': 'Win',
   'fight date': '2010-02-20'},
  {'Billing': 'Preliminary Card',
   'Class weight': '265 lbs',
   'Dec

In [29]:
fighters = []

In [36]:
def get_information_about_all_fighters(tapology_links: list):
    for offset,link in enumerate(tapology_links,901):
        print(link)
        fighters.append(get_fighters_info(link))
        print(offset,'\n')
    return fighters

In [37]:
get_information_about_all_fighters(list_of_fighters[901:])

https://www.tapology.com/fightcenter/fighters/joey-gilbert
901 

https://www.tapology.com/fightcenter/fighters/townsend-saunders
902 

https://www.tapology.com/fightcenter/fighters/8983-john-albert-prince
903 

https://www.tapology.com/fightcenter/fighters/40994-brian-ortega
904 

https://www.tapology.com/fightcenter/fighters/pete-spratt-secret-weapon
905 

https://www.tapology.com/fightcenter/fighters/darren-uyenoyama-bc
906 

https://www.tapology.com/fightcenter/fighters/jason-miller-mayhem
907 

https://www.tapology.com/fightcenter/fighters/guang-you-ning-smasher
908 

https://www.tapology.com/fightcenter/fighters/tyson-griffin
909 

https://www.tapology.com/fightcenter/fighters/adriano-santos
910 

https://www.tapology.com/fightcenter/fighters/50039-jamie-moyle
911 

https://www.tapology.com/fightcenter/fighters/yui-chul-nam
912 

https://www.tapology.com/fightcenter/fighters/16241-nordine-taleb
913 

https://www.tapology.com/fightcenter/fighters/17142-dileno-lopes
914 

https://ww

1014 

https://www.tapology.com/fightcenter/fighters/6617-edinaldo-oliveira-lula
1015 

https://www.tapology.com/fightcenter/fighters/gilbert-melendez-el-nino
1016 

https://www.tapology.com/fightcenter/fighters/12835-jose-maria-tome-no-chance
1017 

https://www.tapology.com/fightcenter/fighters/andre-roberts-the-chief
1018 

https://www.tapology.com/fightcenter/fighters/jared-hamman
1019 

https://www.tapology.com/fightcenter/fighters/29427-edmilson-souza-kevin
1020 

https://www.tapology.com/fightcenter/fighters/danny-martinez-the-gremlin
1021 

https://www.tapology.com/fightcenter/fighters/22461-nico-price
1022 

https://www.tapology.com/fightcenter/fighters/17554-vinc-pichel
1023 

https://www.tapology.com/fightcenter/fighters/tai-bowden
1024 

https://www.tapology.com/fightcenter/fighters/jason-day-dooms
1025 

https://www.tapology.com/fightcenter/fighters/16217-matt-van-buren
1026 

https://www.tapology.com/fightcenter/fighters/11386-pat-schilling-thrilling
1027 

https://www.tap

1125 

https://www.tapology.com/fightcenter/fighters/39227-rose-namajunas-the-riveter
1126 

https://www.tapology.com/fightcenter/fighters/douglas-evans
1127 

https://www.tapology.com/fightcenter/fighters/9048-phil-harris-billy
1128 

https://www.tapology.com/fightcenter/fighters/dale-hartt-the-demon
1129 

https://www.tapology.com/fightcenter/fighters/15366-ben-nguyen
1130 

https://www.tapology.com/fightcenter/fighters/curtis-stout
1131 

https://www.tapology.com/fightcenter/fighters/39472-rob-wilkinson
1132 

https://www.tapology.com/fightcenter/fighters/steve-bosse
1133 

https://www.tapology.com/fightcenter/fighters/jack-nilson-the-ripper
1134 

https://www.tapology.com/fightcenter/fighters/130802-sumidaerji
1135 

https://www.tapology.com/fightcenter/fighters/73587-daniel-spitz
1136 

https://www.tapology.com/fightcenter/fighters/43265-livia-renata-souza
1137 

https://www.tapology.com/fightcenter/fighters/dustin-hazelett-mclovin
1138 

https://www.tapology.com/fightcenter/fight

1235 

https://www.tapology.com/fightcenter/fighters/koji-oishi
1236 

https://www.tapology.com/fightcenter/fighters/nick-agallar
1237 

https://www.tapology.com/fightcenter/fighters/donald-cerrone-cowboy
1238 

https://www.tapology.com/fightcenter/fighters/53808-claudio-puelles
1239 

https://www.tapology.com/fightcenter/fighters/97013-joel-alvarez
1240 

https://www.tapology.com/fightcenter/fighters/marcelo-mello
1241 

https://www.tapology.com/fightcenter/fighters/20241-jodie-esquibel
1242 

https://www.tapology.com/fightcenter/fighters/travis-lutter-the-serial-killer
1243 

https://www.tapology.com/fightcenter/fighters/jason-lambert-the-punisher
1244 

https://www.tapology.com/fightcenter/fighters/60855-naoki-inoue
1245 

https://www.tapology.com/fightcenter/fighters/56388-ji-yun-kim
1246 

https://www.tapology.com/fightcenter/fighters/134267
1247 

https://www.tapology.com/fightcenter/fighters/13882-Sheila-Gaff-Bulldog
1248 

https://www.tapology.com/fightcenter/fighters/jeff-houg

1346 

https://www.tapology.com/fightcenter/fighters/sam-adkins-the-experience
1347 

https://www.tapology.com/fightcenter/fighters/89121-adam-yandiev
1348 

https://www.tapology.com/fightcenter/fighters/michael-bisping-the-count
1349 

https://www.tapology.com/fightcenter/fighters/14588-marlon-moraes
1350 

https://www.tapology.com/fightcenter/fighters/60336-james-mulheron
1351 

https://www.tapology.com/fightcenter/fighters/diego-sanchez-nightmare
1352 

https://www.tapology.com/fightcenter/fighters/leigh-remedios-bulldog
1353 

https://www.tapology.com/fightcenter/fighters/ebenezer-fontes-braga-pitbull
1354 

https://www.tapology.com/fightcenter/fighters/tulio-palhares
1355 

https://www.tapology.com/fightcenter/fighters/rolles-gracie
1356 

https://www.tapology.com/fightcenter/fighters/scott-junk-punk-haole
1357 

https://www.tapology.com/fightcenter/fighters/lance-gibson-fearless
1358 

https://www.tapology.com/fightcenter/fighters/demian-maia
1359 

https://www.tapology.com/fight

...

1567 

https://www.tapology.com/fightcenter/fighters/kenny-robertson
1568 

https://www.tapology.com/fightcenter/fighters/13786-mark-godbeer-the-hand-of
1569 

https://www.tapology.com/fightcenter/fighters/19478-brian-houston-b-hue
1570 

https://www.tapology.com/fightcenter/fighters/61870-shane-young
1571 

https://www.tapology.com/fightcenter/fighters/17046-walter-harris
1572 

https://www.tapology.com/fightcenter/fighters/41835-massio-fullen
1573 

https://www.tapology.com/fightcenter/fighters/29349-santiago-ponzinibbio-el-rasta
1574 

https://www.tapology.com/fightcenter/fighters/matt-arroyo-no-regard
1575 

https://www.tapology.com/fightcenter/fighters/44543-lina-akhtar-lansberg
1576 

https://www.tapology.com/fightcenter/fighters/court-mcgee-the-crusher
1577 

https://www.tapology.com/fightcenter/fighters/luke-zachrich
1578 

https://www.tapology.com/fightcenter/fighters/31763-leandro-silva-buscape
1579 

https://www.tapology.com/fightcenter/fighters/12196-guido-canetti-ninja
158

1679 

https://www.tapology.com/fightcenter/fighters/charlie-valencia
1680 

https://www.tapology.com/fightcenter/fighters/koji-kitao
1681 

https://www.tapology.com/fightcenter/fighters/15647-emily-kagen
1682 

https://www.tapology.com/fightcenter/fighters/tom-blackledge
1683 

https://www.tapology.com/fightcenter/fighters/kevin-randleman-the-monster
1684 

https://www.tapology.com/fightcenter/fighters/keith-rockel
1685 

https://www.tapology.com/fightcenter/fighters/43005-joey-gomez
1686 

https://www.tapology.com/fightcenter/fighters/colin-robinson-big-c
1687 

https://www.tapology.com/fightcenter/fighters/tom-deblass
1688 

https://www.tapology.com/fightcenter/fighters/32750-grant-dawson
1689 

https://www.tapology.com/fightcenter/fighters/14128-Ronny-Markes
1690 

https://www.tapology.com/fightcenter/fighters/17551-pingyuan-liu
1691 

https://www.tapology.com/fightcenter/fighters/matt-horwich-suave
1692 

https://www.tapology.com/fightcenter/fighters/ben-rothwell-northstar
1693 



1792 

https://www.tapology.com/fightcenter/fighters/vernon-white-tiger
1793 

https://www.tapology.com/fightcenter/fighters/keita-nakamura-k-taro
1794 

https://www.tapology.com/fightcenter/fighters/17546-byron-bloodworth
1795 

https://www.tapology.com/fightcenter/fighters/7793-anistavo-gasparzinho
1796 

https://www.tapology.com/fightcenter/fighters/4763-tor-troeng-the-hammer
1797 

https://www.tapology.com/fightcenter/fighters/60294-irene-aldana-robles
1798 

https://www.tapology.com/fightcenter/fighters/fred-ettish
1799 

https://www.tapology.com/fightcenter/fighters/115752-arman-tsarukyan
1800 

https://www.tapology.com/fightcenter/fighters/erick-silva-indio
1801 

https://www.tapology.com/fightcenter/fighters/40903-danielle-taylor
1802 

https://www.tapology.com/fightcenter/fighters/ronald-jhun-machine-gun
1803 

https://www.tapology.com/fightcenter/fighters/rob-yundt
1804 

https://www.tapology.com/fightcenter/fighters/13906-Craig-White
1805 

https://www.tapology.com/fightcent

1904 

https://www.tapology.com/fightcenter/fighters/michael-mcdonald-mayday
1905 

https://www.tapology.com/fightcenter/fighters/49979-khalid-taha
1906 

https://www.tapology.com/fightcenter/fighters/kenichi-yamamoto
1907 

https://www.tapology.com/fightcenter/fighters/felice-herrig-lil-bulldog
1908 

https://www.tapology.com/fightcenter/fighters/jerrod-sanders
1909 

https://www.tapology.com/fightcenter/fighters/18435-clay-collard
1910 

https://www.tapology.com/fightcenter/fighters/keith-mielke
1911 

https://www.tapology.com/fightcenter/fighters/kerry-schall-meat-truck
1912 

https://www.tapology.com/fightcenter/fighters/eric-schafer-red
1913 

https://www.tapology.com/fightcenter/fighters/yoshiyuki-yoshida-zenko
1914 

https://www.tapology.com/fightcenter/fighters/54500-alexis-dufresne
1915 

https://www.tapology.com/fightcenter/fighters/22883-michinori-tanaka
1916 

https://www.tapology.com/fightcenter/fighters/25143-bec-hyatt-rowdy
1917 

https://www.tapology.com/fightcenter/fig

[{'Name': 'Christian Colombo',
  'Nickame': 'Godzilla',
  'Birth date': '1980-01-07',
  'Given name': '',
  'Height': '196',
  'Reach': '204',
  'Record': {'nc': '0', 'wins': '8', 'loses': '4', 'draws': '1'},
  'Last fight': '2018-06-09',
  'Affiliation': 'CSA.dk',
  'Born': 'Denmark',
  'Fighting out of': 'Copenhagen, Denmark',
  'bouts': [{'Billing': '',
    'Class weight': '',
    'Decision': '',
    'Duration': '',
    'Fight under': 'Denmark Regional',
    "Fighter's record": {'wins': '0', 'loses': '0', 'draws': '0'},
    'Injuries': '',
    'Odds': '',
    "Opponent's name": 'Tommy Rasmussen',
    "Opponent's record": {'wins': '0', 'loses': '4', 'draws': '0'},
    'Pay': '',
    'Referee': '',
    'Round': '',
    'Time': '',
    'Title in a fight': '',
    'Way': '',
    'Weigh-In': '',
    'Weightclass': '',
    'Weightclass title name': '',
    'Win or Lose': 'Win',
    'fight date': '2010-02-13'},
   {'Billing': '',
    'Class weight': '',
    'Decision': '',
    'Duration': 

In [8]:
import json

In [9]:
fighters = json.load(open('fighters_json.txt','r'))

In [41]:
json.dump(fighters,open('fighters_25-05-19.txt','w'))

In [11]:
to_update = ["https://www.tapology.com/fightcenter/fighters/106437-viviane-araujo-vivi",
"https://www.tapology.com/fightcenter/fighters/114251-luana-carolina-dread",
"https://www.tapology.com/fightcenter/fighters/55638-carlos-huanchin",
"https://www.tapology.com/fightcenter/fighters/antonio-rogerio-nogueira-minotoro"]

In [15]:
def get_information_about_all_fighters(tapology_links: list):
    for offset,link in enumerate(tapology_links):
        print(link)
        fighters.append(get_fighters_info(link))
        print(offset,'\n')
    return fighters

In [ ]:
fighters = json.load(open('fighters_17-05-19.txt','r'))

In [ ]:
get_information_about_all_fighters(to_update)

https://www.tapology.com/fightcenter/fighters/106437-viviane-araujo-vivi
0 

https://www.tapology.com/fightcenter/fighters/114251-luana-carolina-dread
